<a href="https://colab.research.google.com/github/Mulac/TDA-SentimentAnalysis/blob/master/transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transformer Model**

In [ ]:
!pip install transformers

In [ ]:
import torch
from torchtext import data
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.optim as optim
from time import perf_counter
import pandas as pd 
from nltk.corpus import stopwords

In [ ]:
tokeniser = BertTokenizer.from_pretrained('bert-base-cased')

# Creating the dataset

In [ ]:
def preprocess_tweet(tweet):
  ## TO BE IMPLEMENTED
  return tokeniser.convert_tokens_to_ids(tweet)

In [ ]:
""" 
Defines the fields for the dataset.
The tweets will be tokenised and preprocessed here.
"""

TEXT = data.Field(tokenize = tokeniser.tokenize,
                  sequential = True,
                  use_vocab = False,
                  batch_first = True,
                  preprocessing = preprocess_tweet,
                  init_token = tokeniser.cls_token_id,
                  eos_token = tokeniser.eos_token_id,
                  pad_token = tokeniser.pad_token_id,
                  unk_token = tokeniser.unk_token_id)

LABEL = data.LabelField(dtype = torch.long)

fields = [('label', LABEL), ('tweet', TEXT)]

In [ ]:
"""
Creates dataset and splits it into training data and validation data.
"""

!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/Mulac/TDA-SentimentAnalysis/master/data/0.75_min_confidence.csv

tweet_data = data.TabularDataset(path = '0.75_min_confidence.csv', format = "csv", fields = fields, skip_header = True)
train_data, valid_data = tweet_data.split()

In [ ]:
print("Training data size:", len(train_data))
print("Training data size:", len(valid_data), "\n")

print("Row/Example object in the training data set:", vars(train_data[0]))
print("Converting the tweet text back into tokens gives:", tokeniser.convert_ids_to_tokens(vars(train_data[0])['tweet']), "\n")

print("Row/Example object in the validation data set:", vars(valid_data[0]))
print("Converting the tweet text back into tokens gives:", tokeniser.convert_ids_to_tokens(vars(valid_data[0])['tweet']))

Training data size: 7323
Training data size: 3138 

Row/Example object in the training data set: {'label': 'negative', 'tweet': [137, 10280, 1577, 112, 189, 1138, 3566, 20147, 1146, 1412, 3868, 4169, 1190, 1152, 1225, 119, 5749, 1111, 1217, 1216, 170, 6434, 8694, 119, 108, 9333]}
Converting the tweet text back into tokens gives: ['@', 'united', 'couldn', "'", 't', 'have', 'possibly', 'messed', 'up', 'our', 'trip', 'anymore', 'than', 'they', 'did', '.', 'Thanks', 'for', 'being', 'such', 'a', 'terrible', 'airline', '.', '#', 'disappointed'] 

Row/Example object in the validation data set: {'label': 'positive', 'tweet': [137, 1237, 1592, 3161, 5749, 106, 1124, 1110, 119]}
Converting the tweet text back into tokens gives: ['@', 'American', '##A', '##ir', 'Thanks', '!', 'He', 'is', '.']


In [ ]:
"""
Builds the vocab object for the tweet and label fields in the training data.
"""
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [ ]:
print("Mapping of tokens to numbers for the labels:", LABEL.vocab.stoi)

Mapping of tokens to numbers for the labels: defaultdict(<function _default_unk_index at 0x7f08453beae8>, {'negative': 0, 'neutral': 1, 'positive': 2})


In [ ]:
"""
Splitting the dataset into training and validation.
Sorted by length of tweets, so each batch has similarly sized tweets.
"""
train_iter, valid_iter = data.BucketIterator.splits(
    (train_data, valid_data), sort_key=lambda x: len(x.tweet), batch_size=32)

In [ ]:
print("Number of batches (Training set):", len(train_iter), "\n")
print("Number of batches (Validation set):",len(valid_iter))

Number of batches (Training set): 229 

Number of batches (Validation set): 99


# Building the model

To get the embeddings for the tweets, we will use BERT (pre-trained **Bidirectional Encoder Representations from Transformers** by Google). These embeddings will be passed to the GRU (Gated Recurrent Unit).

In [ ]:
bert = BertModel.from_pretrained('bert-base-cased')

In [ ]:
class Network(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, num_layers, dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size'] ## # size of input features for the GRU = embedding dimension size
        
        self.rnn = nn.GRU(input_size = embedding_dim, 
                          hidden_size = hidden_dim, 
                          num_layers = num_layers, 
                          bidirectional = True, 
                          batch_first = True, 
                          dropout = dropout)
        
        self.lin = nn.Linear(hidden_dim * 2, output_dim) #  bidirectional, so the hidden_size must be multiplied by 2 for the forward and backward pass
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tweet):
                        
        with torch.no_grad():            # don't want to train BERT
            embedded = self.bert(tweet)[0]  ## gets embedding from BERT
        
        output_shape, hidden_state = self.rnn(embedded)   ## pass embedding into GRU
        
        hidden_state = self.dropout(torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)) ## bidirectional, so there are two hidden states

        output = self.lin(hidden_state)  ## final linear layer gives 3 outputs for each tweet in the batch, representing negative, neutral, positive probabilities
        
        return output

# Hyper-parameters and creating the model


In [ ]:
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT_PROB = 0.25
OUTPUT_DIM= 3

model = Network(bert, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, DROPOUT_PROB )

We don't want to train the BERT model, so we set *requires_grad = false* for BERT's parameters.

In [ ]:
for name, parameter in model.named_parameters():
  if name.startswith('bert'):
    parameter.requires_grad = False

# Training the model

In [ ]:
optimiser = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss() ## multiple classes, so cross entropy loss is appropriate

In [ ]:
"""
Training loop
"""
def train():

  model.train()
  correct = 0
  total = 0

  for batch in train_iter:    

    optimiser.zero_grad() 

    output = model(batch.tweet)

    loss = criterion(output, batch.label)
    loss.backward()

    optimiser.step()

    predictions = torch.argmax(output, dim=1) ## gets index of the highest probability sentiment for each tweet

    total += len(batch.label)
    correct += (predictions == batch.label).sum().item()

  print("Training accuracy:", (correct/total)*100)


In [ ]:
def evaluate():

  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for batch in valid_iter:

      output = model(batch.tweet)

      predictions = torch.argmax(output, dim=1)

      total += len(batch.label)
      correct += (predictions == batch.label).sum().item()

  print("Validation accuracy:", (correct/total)*100)


In [20]:
EPOCHS = 5

for epoch in range(EPOCHS):

  print("\nEpoch" ,(epoch+1))
  start = perf_counter()
  train()
  end = perf_counter()
  print("Epoch time:", (end-start)/60, "minutes \n")
  evaluate()


torch.save(model, "saved_transformer_model.pt")


Epoch 1
Training accuracy: 78.66994401201694
Epoch time: 19.650192041966665 minutes 

Validation accuracy: 81.35755258126196

Epoch 2
Training accuracy: 86.15321589512494
Epoch time: 19.3949206674 minutes 

Validation accuracy: 86.26513702995538

Epoch 3
Training accuracy: 87.84651099276253
Epoch time: 19.60196131818334 minutes 

Validation accuracy: 85.65965583173997

Epoch 4
Training accuracy: 89.26669397787792
Epoch time: 19.58538397521667 minutes 

Validation accuracy: 86.36073932441045

Epoch 5
Training accuracy: 90.11334152669671
Epoch time: 19.626577686783335 minutes 

Validation accuracy: 87.31676226896113
